# Predicting swell

1. Data
  * Wave parameters
  * Spectra

1. Data preparation
1. Models
 * GRU
 * CNN1D
 * ResNet
 * Dense
 * Ensemble

1. S3 options
1. Early stopping
1. Learning Area Reduction
1. Running in BoM